# Filtros

Procedimiento de filtrado simple de señales en base a filtros paso bajo (Low-Pass Filter), paso alto (High-Pass Filter), paso banda (Band-Pass Filter) y banda eliminada (Band-Stop Filter). 
Para el filtrado, en lugar de solicitar las curvas de coeficientes del filtro, meramente se pedirán los umbrales precisos, que definirán separaciones entre subarrays de 1s y 0s, valores únicos de coeficientes permitidos (se trata, por tanto, de un filtrado teórico ideal). 

In [14]:
# Imports
from scipy.io import wavfile
import os
import numpy as np
from pathlib import Path

# Inputs
PATH = 'C:\\Users\\Javier\\Desktop\\TFG\\Filtros'      # Path of project's directory
AUDIO_FILE = 'DoM-piano.wav'                                         # Audio file's name
WINDOW_SIZE_SECS = 0.2                                               # Size of the fft window in seconds
OVERLAPPING_SECS = WINDOW_SIZE_SECS / 2                              # Window's overlapping in seconds
FILTER_TYPE = 'BSF'                                                # Filter type in [LPF,HPF,BPF,BSF]
THRESHOLD1 = 400                                                     # First threshold for the filter in Hz
THRESHOLD2 = 600                                                     # Second threshold for the filter (if necessary) in Hz

# Global variables 
SAMPLE_RATE, data = wavfile.read(os.path.join(PATH,AUDIO_FILE))      # Get sample rate (samples per second) and signal data
signal = data if data.ndim == 1 else data.T[0]                       # Get the first channel
WINDOW_SIZE_SAMPLES = int(SAMPLE_RATE * WINDOW_SIZE_SECS)            # Size of the fft window in samples
OVERLAPPING_SAMPLES = int(SAMPLE_RATE * OVERLAPPING_SECS)            # Size of overlapping in samples
AUDIO_SIZE_SECS = len(signal) / SAMPLE_RATE                          # Size of the audio file in seconds
hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

# Files' statistics
print("Sample rate: " + str(SAMPLE_RATE) + " Hz")                   
print("Signal: " + str(signal))                                      
print("Window size: " + str(WINDOW_SIZE_SECS) + " s = " + str(WINDOW_SIZE_SAMPLES) + " samples")
print("Overlapping: " + str(OVERLAPPING_SECS) + " s = " + str(OVERLAPPING_SAMPLES) + " samples")
print("Audio length: " + str(AUDIO_SIZE_SECS) + " s")

# Functions
def extract_window(audio, window_number):                                   # Returns samples of window number <window-number> and true or false whether it's the last window 
    begin = window_number * (WINDOW_SIZE_SAMPLES - OVERLAPPING_SAMPLES)
    end = begin + WINDOW_SIZE_SAMPLES
    
    if end < len(signal): # Commonly
        return False, audio[begin:end]
    else: # The window surpasses the audio data => Complete last elements of the window with zeros
        return True, np.concatenate([audio[begin:len(signal)-1],np.zeros(end-len(signal)+1,dtype=float)])
    
def analysis(window): # Compute the FFT's module curve and return x and y axes in a tuple
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    fft = np.fft.rfft(window) # Evaluations of those frequencies

    return (freqs,fft)

def filter_window(fft): # Filter the FFT's module function according to the selected filter and thresholds
    if FILTER_TYPE == 'LPF':
        filtered = [fft[1][i] if fft[0][i] <= THRESHOLD1 else 0 for i in range(0,len(fft[0]))]
    elif FILTER_TYPE == 'HPF':
        filtered = [fft[1][i] if fft[0][i] >= THRESHOLD1 else 0 for i in range(0,len(fft[0]))]
    elif FILTER_TYPE == 'BPF':
        filtered = [fft[1][i] if fft[0][i] >= THRESHOLD1 and fft[0][i] <= THRESHOLD2 else 0 for i in range(0,len(fft[0]))]
    elif FILTER_TYPE == 'BSF':
        filtered = [fft[1][i] if fft[0][i] <= THRESHOLD1 or fft[0][i] >= THRESHOLD2 else 0 for i in range(0,len(fft[0]))]
    else:
        filtered = fft[1].copy()

    return filtered

def synthesis(ffts): # Synthesize the new signal via the list of windows considering overlapping
    new_signal = (np.real(np.fft.irfft(ffts[0])) * hanning).astype(data.dtype)
    for i in range(1,len(ffts)):
        window = (np.real(np.fft.irfft(ffts[i])) * hanning).astype(data.dtype)
        for j in range(0,OVERLAPPING_SAMPLES):
            new_signal[-OVERLAPPING_SAMPLES+j] = new_signal[-OVERLAPPING_SAMPLES+j] + window[j]
        new_signal = np.append(new_signal,window[OVERLAPPING_SAMPLES:])
    
    return new_signal

def filtering():
    filtered_windows = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal,window_number)
        window_number += 1
        filtered_windows.append(filter_window(analysis(window)))
    
    filtered_signal = synthesis(filtered_windows)
    wavfile.write(os.path.join(PATH,Path(AUDIO_FILE).stem+FILTER_TYPE+'.wav'),SAMPLE_RATE,filtered_signal)

Sample rate: 44100 Hz
Signal: [0.01104736 0.01083374 0.01083374 ... 0.         0.         0.        ]
Window size: 0.2 s = 8820 samples
Overlapping: 0.1 s = 4410 samples
Audio length: 13.933469387755101 s


<ipython-input-14-4bbb7648d514>:17: WavFileWarning: Chunk (non-data) not understood, skipping it.
  SAMPLE_RATE, data = wavfile.read(os.path.join(PATH,AUDIO_FILE))      # Get sample rate (samples per second) and signal data


In [15]:
filtering()